In [22]:
# prompt: hugging face cli python

!pip install -q huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

#Install dependecies

In [1]:
!pip install --upgrade datasets huggingface_hub fsspec transformers torch accelerate
!pip install datasets pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
from datasets import load_dataset
import pandas as pd

In [13]:
dataset = load_dataset("AtlasPolat/yks2024", streaming=False)

In [14]:
df = pd.DataFrame(dataset)



In [15]:
df.size
df.head()

,train
0,{'section': 'TURK-DILI-VE-EDEBIYATI-SOSYAL-BIL...
1,{'section': 'TURK-DILI-VE-EDEBIYATI-SOSYAL-BIL...
2,{'section': 'TURK-DILI-VE-EDEBIYATI-SOSYAL-BIL...
3,{'section': 'TURK-DILI-VE-EDEBIYATI-SOSYAL-BIL...
4,{'section': 'TURK-DILI-VE-EDEBIYATI-SOSYAL-BIL...


In [6]:
# prompt: print(pd['train'][0]['section']) I want to know the number of each section
sec_dict = {}

for i in range(len(pd['train'])):
    if pd['train'][i]['section'] not in sec_dict:
        sec_dict[pd['train'][i]['section']] = 1
    else:
        sec_dict[pd['train'][i]['section']] += 1

print(sec_dict)

{'TURK-DILI-VE-EDEBIYATI-SOSYAL-BILIMLER-1-TESTI': 40, 'SOSYAL-BILIMLER-2-TESTI': 46, 'MATEMATIK-TESTI': 40, 'FEN-BILIMLERI-TESTI': 40, 'TURKCE-TESTI': 40, 'SOSYAL-BILIMLER-TESTI': 25, 'TEMEL-MATEMATIK-TESTI': 40, 'TEMEL-FEN-BILIMLERI-TESTI': 20}


In [24]:
!pip install --upgrade transformers

In [27]:
from huggingface_hub import delete_repo_cache

# This will delete the cache for a specific repository.
# You'll need the repo ID. For Qwen/Qwen3-8B, the repo ID is 'Qwen/Qwen3-8B'.
try:
    delete_repo_cache('Qwen/Qwen3-8B')
    print("Cache for Qwen/Qwen3-8B deleted.")
except Exception as e:
    print(f"Could not delete repo cache: {e}")

ImportError: cannot import name 'delete_repo_cache' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)

In [25]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B" # Changed back to Qwen3-8B

# Check if CUDA is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    # load the tokenizer and the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
    print(f"Model {model_name} loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have a stable internet connection and the model name is correct.")
    print("If you are on a limited resource environment, this model is quite large.")

# Qwen models usually handle padding well, but setting it explicitly can be good.
# For Qwen3, the specific pad token might be different or handled internally by apply_chat_template.
# It's often recommended to use the eos_token as the pad_token if no specific pad_token is set.
if tokenizer.pad_token is None:
    if tokenizer.eos_token:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id
        print(f"Set pad_token to eos_token ({tokenizer.eos_token})")
    else:
        print("Warning: tokenizer has no pad_token and no eos_token. Generation might be problematic.")

Using device: cuda
Error loading model: Could not find Qwen3ForCausalLM neither in <module 'transformers.models.qwen3' from '/usr/local/lib/python3.11/dist-packages/transformers/models/qwen3/__init__.py'> nor in <module 'transformers' from '/usr/local/lib/python3.11/dist-packages/transformers/__init__.py'>!
Please ensure you have a stable internet connection and the model name is correct.
If you are on a limited resource environment, this model is quite large.


In [18]:
import re # For parsing the final answer

results_list = []
think_token_id = 151668 # </think> token ID for Qwen/Qwen3-8B

# Assuming your questions are in df['train']
# Adjust this if your data structure is different
if 'train' in df:
    question_data = df['train']
    print(f"Processing {len(question_data)} questions from df['train']...")
elif 'train' in dataset: # Fallback if df['train'] was not created as expected
    try:
        question_data = [item for item in dataset['train']] # Ensure it's a list
        print(f"Processing {len(question_data)} questions from dataset['train']...")
    except TypeError:
        print("dataset['train'] is not iterable. Please ensure it's loaded correctly.")
        question_data = []
else:
    print("Could not find 'train' data in `df` or `dataset`. Please check your data loading steps.")
    question_data = []


for i, item in enumerate(question_data):
    question_text = item.get("question", "")
    passage = item.get("passage", "")
    choices = item.get("choices", [])

    # Construct the prompt
    prompt_parts = []
    if passage:
        prompt_parts.append(f"Passage: {passage}")
    prompt_parts.append(f"Question: {question_text}")
    prompt_parts.append("Choices:")
    for idx, choice in enumerate(choices):
        prompt_parts.append(f"{idx}. {choice.strip()}")

    prompt_parts.append("\nBased on the passage and question, which choice is correct? Your answer should be only a single digit: 0, 1, 2, 3, or 4, corresponding to the choice index.")

    full_prompt = "\n".join(prompt_parts)

     # Define the new system prompt with the Turkish instruction and few-shot example
    turkish_instruction = "Parçaya ve soruya göre hangi seçenek doğrudur? Cevabınız sadece seçeneğin indeksine karşılık gelen tek bir rakam (0, 1, 2, 3 veya 4) olmalıdır."

    system_prompt_content = f"""You are an AI assistant. That would help users answer questions based on provided passages and choices. The language of the questions and passages is Turkish. And you should provide your answer as a single digit corresponding to the index of the correct choice.
{turkish_instruction}
Analyze the question and provide your thinking process before giving the final answer as a single digit.

Here is an example of the input format you will receive and the thinking process/output format you should follow:

Input Example (User will provide this structure):
Passage: Kitap Sanat, edebiyat ve eleştirinin kökeninin yakın dönemlere dayandığını iddia ederken resim, şiir ve müzik türlerinin izlerine daha önceki çağlarda rastlanmadığını değil; bu türlere bakışımızın önceki dönemlerden farklı olduğunu ileri sürüyorum. Söz gelimi İlyada destanı 2000’li yıllarda Arkaik Çağ’dakinden çok farklı işlevler görür. Modern okurlar için bir başyapıt olarak Batı edebiyatında önemli bir yeri vardır. Ama Antik Yunan’da yaşayanlar, onun edebiyat olduğunu düşünemezlerdi çünkü kavramsal olarak henüz böyle bir sınıflama yoktu. Bu epik şiir, kurmaca olması yönüyle benzersiz bir yazın tarzı payesi almak şöyle dursun, toplumsal hayatla sıkı sıkıya bütünleşmişti; törenlerde okunuyor, öğretiliyor ve sık sık hukuksal anlaşmazlıkların karara bağlanmasında kullanılıyordu.
Question: Bu parçaya göre Antik Yunan’da İlyada destanının günümüzdeki gibi bir edebî yapıt olarak görülmemesinin nedeni aşağıdakilerden hangisidir?
Choices:
0. Sözlü kültür geleneğinden beslendiği için toplumun ürünü olarak algılanması
1. Sosyal yaşamda bir amaca hizmet ettiği için faydacı yaklaşımla incelenmesi
2. Hukuki metinlerin boşluğunu doldurduğu için estetikyönüne odaklanılmaması
3. Yazınsal metin kategorisi oluşmadığından bu yönünün değerlendirilememesi
4. İçeriğinin, yazıldığı dönemin sosyal ve siyasal işleyişine yönelik tasarlanması

Your Output Example (after your thinking process, which should be enclosed in <think>...</think> tags if the model supports it, or just precede the answer):
<think> The passage states that in Ancient Greece, the concept of 'literature' as a classification did not exist. Therefore, the Iliad could not be seen as a literary work in the way modern readers see it. This directly corresponds to the idea that a 'literary text category' had not yet been formed. Choice 3 reflects this. </think>
3
"""

    messages = [
        {"role": "system", "content": system_prompt_content},
        {"role": "user", "content": full_prompt}
    ]

    try:
        text_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True # Crucial for Qwen3-8B thinking mode
        )
        model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

        # Conduct text completion
        # max_new_tokens might need to be generous to allow for thinking.
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512, # Increased to allow for thinking
            pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
        )

        output_ids_only = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

        thinking_content = ""
        content_after_thinking = ""
        parsed_answer = -1

        try:
            # Find the last occurrence of the </think> token
            # rindex finds the first occurrence from the end
            index_of_think_end = len(output_ids_only) - output_ids_only[::-1].index(think_token_id)

            thinking_output_ids = output_ids_only[:index_of_think_end]
            # The thinking content includes the </think> token itself if we decode up to index_of_think_end
            # To exclude it, decode up to index_of_think_end - 1 (if the token is a single ID)
            # However, Qwen's thinking often has <think>...</think>, so decoding up to index_of_think_end
            # and then stripping might be fine, or more precise slicing is needed if special tokens are involved.
            # Let's decode the thinking part (potentially including the </think> token)
            thinking_content = tokenizer.decode(thinking_output_ids, skip_special_tokens=False).strip()
            # And the content part (after </think>)
            content_after_thinking_ids = output_ids_only[index_of_think_end:]
            content_after_thinking = tokenizer.decode(content_after_thinking_ids, skip_special_tokens=True).strip()

        except ValueError:
            # </think> token not found, assume the whole output is the content
            print(f"Warning: </think> token (ID {think_token_id}) not found in output for question {i+1}.")
            content_after_thinking = tokenizer.decode(output_ids_only, skip_special_tokens=True).strip()
            thinking_content = "[No explicit thinking block found]"

        # Attempt to extract the first digit from the content_after_thinking
        # Using regex to find the first single digit
        match = re.search(r'\b([0-4])\b', content_after_thinking) # Look for a single digit 0-4 as a whole word
        if not match: # If not found as a whole word, try any digit
            match = re.search(r'([0-4])', content_after_thinking)

        if match:
            parsed_answer = int(match.group(1))
        else:
            parsed_answer = -1 # Default if no number 0-4 is found

        print(f"\n--- Question {i+1} ---")
        # print(f"Prompt:\n{full_prompt}") # Can be verbose
        print(f"Thinking Content:\n{thinking_content}")
        print(f"Content After Thinking (raw answer part):\n{content_after_thinking}")
        print(f"Parsed Answer (index): {parsed_answer}")
        print(f"Actual Answer (index): {item.get('answer')}")

        results_list.append({
            "question_number_in_dataset": item.get("question_number"),
            "section": item.get("section"),
            "question": question_text,
            # "passage": passage, # Can be verbose
            # "choices": choices, # Can be verbose
            # "prompt": full_prompt, # Can be verbose
            "thinking_content": thinking_content,
            "raw_content_after_thinking": content_after_thinking,
            "parsed_answer_index": parsed_answer,
            "correct_answer_index": item.get("answer")
        })

    except Exception as e:
        print(f"Error processing question {i+1}: {e}")
        results_list.append({
            "question_number_in_dataset": item.get("question_number"),
            "section": item.get("section"),
            "question": question_text,
            "error": str(e)
        })

    # Optional: break after a few questions for testing
    # if i >= 0: # Process only the first question for quick test
    #     print("Stopping early for testing.")
    #     break

# Convert results to a DataFrame for easier analysis
results_df = pd.DataFrame(results_list)
print("\n\n--- Batch Processing Complete ---")
if not results_df.empty:
    print(results_df.head())
    # Display relevant columns if the DataFrame is large
    if len(results_df.columns) > 5:
        print("\nKey columns from results:")
        print(results_df[['question_number_in_dataset', 'parsed_answer_index', 'correct_answer_index', 'thinking_content']].head())
        #save the results to a CSV file

else:
    print("No results were processed.")

# You can now save or further analyze results_df
# For example: results_df.to_csv("model_answers_qwen3_8b_thinking.csv", index=False)

results_df.to_csv("model_answers_qwen3_8b_thinking.csv", index=False)


# lets print the results_df by comparing the parsed_answer_index and correct_answer_index


dict_results = {}

for index, row in results_df.iterrows():
    # If the question is correctly answere
    # if correcct add to the dict_results the section make it one if the section is not in the dict_results if it is in the dict_results add one to the section
    if row['parsed_answer_index'] == row['correct_answer_index']:
        if row['section'] not in dict_results:
            dict_results[row['section']] = 1
        else:
            dict_results[row['section']] += 1


Processing 291 questions from df['train']...
Error processing question 1: name 'model' is not defined
Error processing question 2: name 'model' is not defined
Error processing question 3: name 'model' is not defined
Error processing question 4: name 'model' is not defined
Error processing question 5: name 'model' is not defined
Error processing question 6: name 'model' is not defined
Error processing question 7: name 'model' is not defined
Error processing question 8: name 'model' is not defined
Error processing question 9: name 'model' is not defined
Error processing question 10: name 'model' is not defined
Error processing question 11: name 'model' is not defined
Error processing question 12: name 'model' is not defined
Error processing question 13: name 'model' is not defined
Error processing question 14: name 'model' is not defined
Error processing question 15: name 'model' is not defined
Error processing question 16: name 'model' is not defined
Error processing question 17: name '

KeyError: 'parsed_answer_index'

In [ ]:
print("Job finished. Disconnecting...")

from google.colab import runtime
runtime.unassign()